In [1]:
%load_ext autoreload

import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import torch
import torch.nn as nn

module_path = os.path.abspath(os.path.join('../../py-conjugated'))
if module_path not in sys.path:
    sys.path.append(module_path)
import morphology_networks as net
import model_training as train
import model_testing as test
import physically_informed_loss_functions as PhysLoss
import network_utils as nuts

torch.manual_seed(28)

In [2]:
train_path = '/Volumes/Tatum_SSD-1/Grad_School/m2py/Morphology_labels/OPV_morph_maps/train_set/'
test_path = '/Volumes/Tatum_SSD-1/Grad_School/m2py/Morphology_labels/OPV_morph_maps/test_set/'

train_dataset = nuts.local_OPV_ImDataset(train_path)
test_dataset = nuts.local_OPV_ImDataset(test_path)

In [3]:
num_epochs = 10
learning_rate = 0.008

In [4]:
input_channels = 2
model = net.OPV_m2py_NN(input_channels)

#define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [5]:
%autoreload

train_epoch_losses = []
test_epoch_losses = []

for epoch in range(num_epochs):
    
    # First Train the image branch
    train_epoch_loss = train.train_OPV_m2py_model(model = model,
                                   training_data_set = train_dataset,
                                   criterion = criterion,
                                   optimizer = optimizer)
    
    train_epoch_losses.append(train_epoch_loss)
    
    test_epoch_loss = test.eval_OPV_m2py_model(model = branch_model,
                                 testing_data_set = test_dataset,
                                 criterion = criterion)
    
    test_epoch_losses.append(test_epoch_loss)

predictions: tensor([ 2.7416, -1.8820], grad_fn=<SqueezeBackward0>)
labels: tensor([ 15., 100.])


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, im_train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, im_test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()